In [11]:
#@title TASK 1

import numpy as np

EMPTY = 0
WHITE = 1
BLACK = 2
BOARD_SIZE = 8

class SimpleCheckers:
    def __init__(self):
        self.board = self.setup_board()
        self.current_player = WHITE
        self.game_over = False

    def setup_board(self):
        board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)

        for row in range(3):
            for col in range(BOARD_SIZE):
                if (row + col) % 2 == 1:
                    board[row][col] = BLACK

        for row in range(5, 8):
            for col in range(BOARD_SIZE):
                if (row + col) % 2 == 1:
                    board[row][col] = WHITE

        return board

    def print_board(self):
        print("  0 1 2 3 4 5 6 7")
        for row in range(BOARD_SIZE):
            print(row, end=" ")
            for col in range(BOARD_SIZE):
                piece = self.board[row][col]
                if piece == WHITE:
                    print("W", end=" ")
                elif piece == BLACK:
                    print("B", end=" ")
                else:
                    print(".", end=" ")
            print()

    def get_moves(self, player):
        moves = []
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                if self.board[row][col] == player:
                    directions = [(-1, -1), (-1, 1)] if player == WHITE else [(1, -1), (1, 1)]
                    for dr, dc in directions:
                        new_row, new_col = row + dr, col + dc
                        if 0 <= new_row < BOARD_SIZE and 0 <= new_col < BOARD_SIZE:
                            if self.board[new_row][new_col] == EMPTY:
                                moves.append(((row, col), (new_row, new_col)))
        return moves

    def make_move(self, start, end):
        start_row, start_col = start
        end_row, end_col = end
        self.board[end_row][end_col] = self.board[start_row][start_col]
        self.board[start_row][start_col] = EMPTY

    def play(self):
        print("Welcome to Simple Checkers!")
        print("You are White (W), AI is Black (B)")
        print("Enter moves as 'start_row start_col end_row end_col'")

        while not self.game_over:
            self.print_board()

            if self.current_player == WHITE:
                moves = self.get_moves(WHITE)
                if not moves:
                    print("No valid moves. You lose!")
                    break

                print("\nYour possible moves:")
                for i, move in enumerate(moves):
                    print(f"{i+1}: {move[0]} -> {move[1]}")

                while True:
                    try:
                        choice = int(input(f"Choose move (1-{len(moves)}): ")) - 1
                        if 0 <= choice < len(moves):
                            self.make_move(moves[choice][0], moves[choice][1])
                            print(f"You moved: {moves[choice][0]} -> {moves[choice][1]}")
                            break
                        print("Invalid choice. Try again.")
                    except ValueError:
                        print("Please enter a number.")
            else:
                moves = self.get_moves(BLACK)
                if not moves:
                    print("AI has no moves. You win!")
                    break

                move = moves[0]
                self.make_move(move[0], move[1])
                print(f"AI moved: {move[0]} -> {move[1]}")

            self.current_player = BLACK if self.current_player == WHITE else WHITE

if __name__ == "__main__":
    game = SimpleCheckers()
    game.play()


Welcome to Simple Checkers!
You are White (W), AI is Black (B)
Enter moves as 'start_row start_col end_row end_col'
  0 1 2 3 4 5 6 7
0 . B . B . B . B 
1 B . B . B . B . 
2 . B . B . B . B 
3 . . . . . . . . 
4 . . . . . . . . 
5 W . W . W . W . 
6 . W . W . W . W 
7 W . W . W . W . 

Your possible moves:
1: (5, 0) -> (4, 1)
2: (5, 2) -> (4, 1)
3: (5, 2) -> (4, 3)
4: (5, 4) -> (4, 3)
5: (5, 4) -> (4, 5)
6: (5, 6) -> (4, 5)
7: (5, 6) -> (4, 7)
Choose move (1-7): 1
You moved: (5, 0) -> (4, 1)
  0 1 2 3 4 5 6 7
0 . B . B . B . B 
1 B . B . B . B . 
2 . B . B . B . B 
3 . . . . . . . . 
4 . W . . . . . . 
5 . . W . W . W . 
6 . W . W . W . W 
7 W . W . W . W . 
AI moved: (2, 1) -> (3, 0)
  0 1 2 3 4 5 6 7
0 . B . B . B . B 
1 B . B . B . B . 
2 . . . B . B . B 
3 B . . . . . . . 
4 . W . . . . . . 
5 . . W . W . W . 
6 . W . W . W . W 
7 W . W . W . W . 

Your possible moves:
1: (4, 1) -> (3, 2)
2: (5, 2) -> (4, 3)
3: (5, 4) -> (4, 3)
4: (5, 4) -> (4, 5)
5: (5, 6) -> (4, 5)
6: (5, 6) -> (

KeyboardInterrupt: Interrupted by user

In [4]:
#@title TASK 2

import math

def alpha_beta(cards, left, right, depth, alpha, beta, maximizing_player):
    if left > right:
        return 0

    if maximizing_player:
        pick_left = cards[left] + alpha_beta(cards, left + 1, right, depth + 1, alpha, beta, False)
        pick_right = cards[right] + alpha_beta(cards, left, right - 1, depth + 1, alpha, beta, False)
        best = max(pick_left, pick_right)
        alpha = max(alpha, best)
        if beta <= alpha:
            return best
        return best
    else:
        if cards[left] < cards[right]: #Min choose small value to reduce Max's score
            return alpha_beta(cards, left + 1, right, depth + 1, alpha, beta, True)
        else:
            return alpha_beta(cards, left, right - 1, depth + 1, alpha, beta, True)

def play_game(cards):
    max_score = 0
    min_score = 0
    left = 0
    right = len(cards) - 1

    print(f"Initial Cards: {cards}")

    turn = 'Max'
    while left <= right:
        if turn == 'Max':
            pick_left = cards[left] + alpha_beta(cards, left + 1, right, 0, -math.inf, math.inf, False)
            pick_right = cards[right] + alpha_beta(cards, left, right - 1, 0, -math.inf, math.inf, False)

            if pick_left >= pick_right:
                chosen = cards[left]
                left += 1
            else:
                chosen = cards[right]
                right -= 1

            max_score += chosen
            print(f"Max picks {chosen}, Remaining Cards: {cards[left:right+1]}")
            turn = 'Min'

        else:
            if cards[left] < cards[right]:
                chosen = cards[left]
                left += 1
            else:
                chosen = cards[right]
                right -= 1

            min_score += chosen
            print(f"Min picks {chosen}, Remaining Cards: {cards[left:right+1]}")
            turn = 'Max'

    print(f"\nFinal Scores - Max: {max_score}, Min: {min_score}")
    if max_score > min_score:
        print("Winner: Max")
    elif max_score < min_score:
        print("Winner: Min")
    else:
        print("It’s a Draw!")

cards = [4, 10, 6, 2, 9, 5]
play_game(cards)


Initial Cards: [4, 10, 6, 2, 9, 5]
Max picks 5, Remaining Cards: [4, 10, 6, 2, 9]
Min picks 4, Remaining Cards: [10, 6, 2, 9]
Max picks 10, Remaining Cards: [6, 2, 9]
Min picks 6, Remaining Cards: [2, 9]
Max picks 9, Remaining Cards: [2]
Min picks 2, Remaining Cards: []

Final Scores - Max: 24, Min: 12
Winner: Max


In [6]:
#@title TASK 3

import random

GRID_SIZE = 10
SHIP_SIZES = [5, 4, 3, 3, 2]

EMPTY = '.'
SHIP = 'S'
HIT = 'X'
MISS = 'O'

def create_grid():
    return [[EMPTY for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

def place_ship_randomly(grid, size):
    while True:
        orientation = random.choice(['H', 'V'])
        if orientation == 'H':
            row = random.randint(0, GRID_SIZE - 1)
            col = random.randint(0, GRID_SIZE - size)
            if all(grid[row][col+i] == EMPTY for i in range(size)):
                for i in range(size):
                    grid[row][col+i] = SHIP
                return
        else:
            row = random.randint(0, GRID_SIZE - size)
            col = random.randint(0, GRID_SIZE - 1)
            if all(grid[row+i][col] == EMPTY for i in range(size)):
                for i in range(size):
                    grid[row+i][col] = SHIP
                return

def place_all_ships(grid):
    for size in SHIP_SIZES:
        place_ship_randomly(grid, size)

def print_grid(grid, hide_ships=False):
    print("  " + " ".join("ABCDEFGHIJ"))
    for idx, row in enumerate(grid):
        display_row = []
        for cell in row:
            if hide_ships and cell == SHIP:
                display_row.append(EMPTY)
            else:
                display_row.append(cell)
        print(f"{idx} {' '.join(display_row)}")
    print()

def parse_coordinate(coord):
    if len(coord) < 2 or not coord[0].isalpha() or not coord[1:].isdigit():
        return None
    col = ord(coord[0].upper()) - ord('A')
    row = int(coord[1:])
    if 0 <= row < GRID_SIZE and 0 <= col < GRID_SIZE:
        return row, col
    return None

def attack(grid, row, col):
    if grid[row][col] == SHIP:
        grid[row][col] = HIT
        return "Hit!"
    elif grid[row][col] == EMPTY:
        grid[row][col] = MISS
        return "Miss"
    return "Already tried"

def all_ships_sunk(grid):
    return all(cell != SHIP for row in grid for cell in row)

def get_adjacent_targets(row, col):
    return [(r, c) for r, c in [
        (row-1, col), (row+1, col), (row, col-1), (row, col+1)
    ] if 0 <= r < GRID_SIZE and 0 <= c < GRID_SIZE]

def ai_choose_target(player_grid, ai_memory):
    while ai_memory['hits']:
        last_hit = ai_memory['hits'][-1]
        targets = get_adjacent_targets(*last_hit)
        random.shuffle(targets)
        for r, c in targets:
            if player_grid[r][c] in [EMPTY, SHIP]:
                return r, c
        ai_memory['hits'].pop()

    while True:
        r = random.randint(0, GRID_SIZE - 1)
        c = random.randint(0, GRID_SIZE - 1)
        if player_grid[r][c] in [EMPTY, SHIP]:
            return r, c

def battleship_game():
    player_grid = create_grid()
    ai_grid = create_grid()
    place_all_ships(player_grid)
    place_all_ships(ai_grid)

    ai_memory = {'hits': []}

    print("Welcome to Battleship!")
    print("Your Grid:")
    print_grid(player_grid)

    while True:
        print("Opponent's Grid:")
        print_grid(ai_grid, hide_ships=True)
        while True:
            user_input = input("Enter attack coordinate (e.g., B4): ")
            coord = parse_coordinate(user_input)
            if coord:
                r, c = coord
                result = attack(ai_grid, r, c)
                print(f"Player attacks: {user_input.upper()} → {result}")
                break
            else:
                print("Invalid input. Try again.")

        if all_ships_sunk(ai_grid):
            print("You win!")
            break

        r, c = ai_choose_target(player_grid, ai_memory)
        result = attack(player_grid, r, c)
        coord_str = f"{chr(c + ord('A'))}{r}"
        print(f"AI attacks: {coord_str} → {result}")
        if result == "Hit!":
            ai_memory['hits'].append((r, c))

        print("Your Grid:")
        print_grid(player_grid)

        if all_ships_sunk(player_grid):
            print("AI wins!")
            break

battleship_game()


Welcome to Battleship!
Your Grid:
  A B C D E F G H I J
0 . . . . S . . . . .
1 . . . . S S S S . .
2 . . . . S . . . . .
3 S . . . . . . . . .
4 S . . . . . . . . .
5 S . . . . . . . . .
6 S S S . . . . . . .
7 S . . . . . . . . .
8 . . . . . . S S S S
9 . . . . . . . . . .

Opponent's Grid:
  A B C D E F G H I J
0 . . . . . . . . . .
1 . . . . . . . . . .
2 . . . . . . . . . .
3 . . . . . . . . . .
4 . . . . . . . . . .
5 . . . . . . . . . .
6 . . . . . . . . . .
7 . . . . . . . . . .
8 . . . . . . . . . .
9 . . . . . . . . . .

Enter attack coordinate (e.g., B4): B4
Player attacks: B4 → Miss
AI attacks: H8 → Hit!
Your Grid:
  A B C D E F G H I J
0 . . . . S . . . . .
1 . . . . S S S S . .
2 . . . . S . . . . .
3 S . . . . . . . . .
4 S . . . . . . . . .
5 S . . . . . . . . .
6 S S S . . . . . . .
7 S . . . . . . . . .
8 . . . . . . S X S S
9 . . . . . . . . . .

Opponent's Grid:
  A B C D E F G H I J
0 . . . . . . . . . .
1 . . . . . . . . . .
2 . . . . . . . . . .
3 . . . . . . . .

KeyboardInterrupt: Interrupted by user